In [ ]:
from sklearn import metrics,preprocessing,model_selection
from sklearn.metrics import accuracy_score
import keras
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import string
import pandas as pd
import re
import spacy
from nltk.corpus import stopwords
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from spacy.lang.en import English
spacy.load('en')
parser = English()

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

embed = hub.Module("../input/deeppavlovrussianelmo/elmo_ru-news_wmt11-16_1.5M_steps", trainable=True)

def ELMoEmbedding(x):
    return embed([x], signature="default", as_dict=True)["default"]

In [ ]:
def encode(le_enc, labels):
    enc = le_enc.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le_enc, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le_enc.inverse_transform(dec)

In [ ]:
trainDF_Sheet_1 = pd.read_csv('../input/ruatd2022/train.csv')

In [ ]:
trainDF_Sheet_1.head(10)

In [ ]:
trainDF_Sheet_1.shape

In [ ]:
trainDF_Sheet_1['Class'].unique()

In [ ]:
trainDF_Sheet_1['Class'].value_counts()

In [ ]:
sns.set(rc={'figure.figsize':(8,8)})
sns.countplot(trainDF_Sheet_1['Class'])

In [ ]:
x_train = trainDF_Sheet_1['Text'].tolist()
y_train = trainDF_Sheet_1['Class'].tolist()

le_enc = preprocessing.LabelEncoder()
le_enc.fit(y_train)

y_train = encode(le_enc, y_train)

In [ ]:
test = pd.read_csv('../input/atdval/val.csv')
x_test = test['Text'].tolist()
y_test = test['Class'].tolist()

y_test = encode(le_enc, y_test)

x_test = np.asarray(x_test)
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [ ]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(2, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=10, batch_size=16)
    model.save_weights('./response-elmo-model.h5')

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./response-elmo-model.h5')  
    predicts = model.predict(x_test, batch_size=16)

In [ ]:
y_test = decode(le_enc, y_test)
y_preds = decode(le_enc, predicts)

In [ ]:
np.save('elmo_model_output.npy', y_preds)

In [ ]:
print(metrics.confusion_matrix(y_test, y_preds))

In [ ]:
print(metrics.classification_report(y_test, y_preds))

In [ ]:
print("Accuracy of ELMO is:",accuracy_score(y_test,y_preds))